In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bashim-quotes/dataset.jsonl


## Задание:
1. Обучить модель GPT для генерации своих цитат
2. взять новостные данные из https://github.com/natasha/corus load_lenta2, нам понадобится сам текст и заголовок. Обучить модель T5/ или GPT для генерации заголовков для статей

### Загрузка библиотек и датасета

In [2]:
!pip install datasets transformers

In [3]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)
import warnings
warnings.filterwarnings("ignore")

In [4]:
model_name = "bankholdup/rugpt3_song_writer"

In [5]:
df_rec = pd.read_json('../input/bashim-quotes/dataset.jsonl', lines=True).set_index('id')

In [6]:
df_rec.head()

,date,rating,text
id,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они..."
2,2004-08-30 11:25:00+00:00,25105.0,<томатик_рад> а ты не чувствуешь красоту мира?...
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаз..."
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать ..."
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с ...


In [7]:
df_rec.shape

(81497, 3)

In [8]:
df_rec.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81497 entries, 1 to 463648
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   date    81497 non-null  datetime64[ns, UTC]
 1   rating  80642 non-null  float64            
 2   text    81497 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), object(1)
memory usage: 2.5+ MB


In [9]:
import re
def clear_text(text):
    clr_text = re.sub(r"<.*?>", " ", text).lower()
    clr_text = summary = re.sub(r"\s", " ", clr_text)
    return clr_text

In [10]:
df_rec["clear_text"] = df_rec["text"].apply(lambda x: clear_text(x))
df_rec.head()

,date,rating,text,clear_text
id,,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они...","ppdv, все юниксы очень дружелюбны.. они прос..."
2,2004-08-30 11:25:00+00:00,25105.0,<томатик_рад> а ты не чувствуешь красоту мира?...,а ты не чувствуешь красоту мира? честно го...
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаз...","""мышка, почему у тебя такие большие глаза?"" ..."
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать ...","""мальчики, вы что больные, бегать в палату к..."
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с ...,мы - как разработчики - живём с субейзом под...


In [30]:
!pip install pymorphy2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 31.1 MB/s eta 0:00:0000:0100:01m


In [39]:
!pip install pymorphy2

import re
import pymorphy2
morph=pymorphy2.MorphAnalyzer()

def text_proc_3(text):
    if text != '':
        text = re.sub(r"\s+", " ", text) # убрал лишние пробелы (\s+ Любой пробельный символ 1 и более вхождений шаблона)
        text = re.sub(r"^\s+", "", text) #  уберу первые пробелы в начале строки
        text = text.rstrip() #  уберу последний пробел в начале строки
    return text

def lemma(txt):
    return morph.parse(txt)[0].normal_form

def concat(vec):
    return ' '.join(vec)

def prep1(string):
    string = string.lower()
    string = string.replace('ё', 'е')
    string = string.replace(':', '.')
    string = string.replace('!', '.')
    string = string.replace(',', '.')
    string = string.replace('.', ' . ')
    string = string.replace(' . ', ' ')
    string = string.replace('№', '')
    string = string.replace('sd', '')
    string = string.replace('im', '')
    string = string.replace('\\', ' ')
    string = string.replace(' .', '. ')
    string = string.replace('•', '.')
    string = string.replace('.', ' ')
    string = string.replace('xxx', ' ').replace('yyy', ' ').replace('ххх', ' ')
    string = re.sub('[^а-яА-Я]', ' ', string) # заменил на пробел не буквы
    string = text_proc_3(concat(list(  [lemma(j) for j in string.split() ] ) ))
    string = text_proc_3(string) # убрал лишние пробелы 
    return string

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [40]:
df_rec['clear_text'] = df_rec['clear_text'].apply(lambda x: prep1(x))

In [41]:
df_rec.loc[463648, 'clear_text'].replace('ххх', ' ')

'вот ребята из английский фирма давный давно в девяностый решить построить такой суперкар чтоб всё охренель купить где то двигнуть и вточить по завет чамберлить поставить на стенд чтоб мощность померить и порвать дино стенд так что хватить уже про свой уникальность энтузиаст весь мир вращать глобус совместно'

### Подготовка к обучению

In [42]:
data = df_rec.loc[:, 'clear_text']
data

id
1         всё юникс очень дружелюбный они просто очень р...
2         а ты не чувствовать красота мир честно говорит...
3         мышка почему у ты такой больший глаз уйти я ха...
4         мальчик вы что больной бегать в палата к девоч...
5         мы как разработчик жить с субейз под одбец хор...
                                ...                        
463644    угадать не гуголь что такой жопиздать намекнут...
463645    посетить шальной мысль заняться себя жирок сбр...
463646    всебожьярос судьба айтишник вообще незавидный ...
463647    прибыть на место правоохранитель установить чт...
463648    вот ребята из английский фирма давный давно в ...
Name: clear_text, Length: 81497, dtype: object

In [43]:
from sklearn.model_selection import train_test_split

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        data += summary + "  "
    f.write(data)

In [44]:
train, test = train_test_split(data, test_size=0.15)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

In [45]:
print("Train dataset length: "+ str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 69272
Test dataset length: 12225


In [46]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bankholdup--rugpt3_song_writer/snapshots/b36e2b2198ad85d47b3685a9340f9d7404153d33/config.json
Model config GPT2Config {
  "_name_or_path": "bankholdup/rugpt3_song_writer",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.26.1",
  "use_cac

In [47]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

Loading features from cached file cached_lm_GPT2TokenizerFast_128_train_dataset.txt [took 0.252 s]
Loading features from cached file cached_lm_GPT2TokenizerFast_128_test_dataset.txt [took 0.031 s]


### Fine-tuning

In [48]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_name)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bankholdup--rugpt3_song_writer/snapshots/b36e2b2198ad85d47b3685a9340f9d7404153d33/config.json
Model config GPT2Config {
  "_name_or_path": "bankholdup/rugpt3_song_writer",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.26.1",
  "use_cac

In [49]:
training_args = TrainingArguments(

    "phrase",
    evaluation_strategy = "epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=1e-4,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    
    )

PyTorch: setting up devices


In [50]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [51]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,4.983200,4.855072
2,4.534500,4.798376
3,4.118600,4.828309


TrainOutput(global_step=19245, training_loss=4.590447980603728, metrics={'train_runtime': 2261.0217, 'train_samples_per_second': 34.044, 'train_steps_per_second': 8.512, 'total_flos': 5028173217792000.0, 'train_loss': 4.590447980603728, 'epoch': 3.0})

In [52]:
def generate_text(prefix):
    tokens = tokenizer(prefix, return_tensors='pt').to('cuda')
    size = tokens['input_ids'].shape[1]

    output = model.generate(
        **tokens, 
        #end_token=end_token_id,
        do_sample=False,
        max_length=size+50, 
        early_stopping=True,
        length_penalty=2.0,
        repetition_penalty=8., 
        temperature=0.5,
        num_beams=3,
        no_repeat_ngram_size=5
    )

    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return prefix + result

In [53]:
print(generate_text("привет!"))

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


привет! у нас тут на работе в туалете висит табличка с надписью не курить  ххх а я вот думаю что делать если при загрузке винды из за того что она перестала загружаться то надо ее переустановить или почистить от вирусов


In [54]:
print(generate_text("Погода была прекрасная"))

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Погода была прекрасная и солнечная а я в это время на работе  ххх у меня есть знакомая которая считает что если человек не может ходить по дому то он ненормальный потому что ему нельзя гулять с незнакомыми людьми которые могут быть опасны для его здоровья но при


In [55]:
print(generate_text("Как сдать экзамен?"))

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Как сдать экзамен?  ххх у нас на работе есть чувак который постоянно ходит в наушниках и слушает все что я ему говорю ну ты же музыкант он не может слушать музыку без наушников а если бы мог то это был бы человек с музыкальным слухом
